In [199]:
import psycopg2
import pandas as pd

In [200]:
#%run functions.ipynb

In [201]:
def do_query(query, print_out = False, commit = False):
    try:
        con = psycopg2.connect(
                    host = "127.0.0.11",
                    database="ZSBD_p1",
                    user = "inzdanych123",
                    password = "inzdanych123",
                    port = "5432")

        cur = con.cursor()

        cur.execute(query)
        
        for notice in con.notices:
            print(notice)

        if commit:
            con.commit()

        if print_out:
            column_names = [desc[0] for desc in cur.description]
            row_list = []
            row = cur.fetchone()
            while row is not None:
                row_list.append(row)
                #print(row)
                row = cur.fetchone()
            df = pd.DataFrame(row_list, columns=column_names)
            print(df)
        
        cur.close()
        con.close()
        print('\n')
    except (Exception, psycopg2.DatabaseError) as error:
            print(error)

    finally:
        if con is not None:
            con.close()

# FUNCTIONS

In [202]:
def function_def(func, commit = False):
    try:
        con = psycopg2.connect(
                    host = "127.0.0.11",
                    database="ZSBD_p1",
                    user = "inzdanych123",
                    password = "inzdanych123",
                    port = "5432")

        cur = con.cursor()

        cur.execute(func)
        if commit:
            con.commit()

        cur.close()
        con.close()

    except (Exception, psycopg2.DatabaseError) as error:
            print(error)

    finally:
        if con is not None:
            con.close()

In [203]:
def function_tester(function, *arguments_of_it):
    try:
        con = psycopg2.connect(
                    host = "127.0.0.11",
                    database="ZSBD_p1",
                    user = "inzdanych123",
                    password = "inzdanych123",
                    port = "5432")

        cur = con.cursor()
        
        if len(arguments_of_it) == 1:
            cur.execute(function + "('" +str(arguments_of_it[0]) + "')")
        else:
            cur.execute(function + str(arguments_of_it))

        column_names = [desc[0] for desc in cur.description]
        row_list = []
        row = cur.fetchone()
        while row is not None:
            row_list.append(row)
            #print(row)
            row = cur.fetchone()
        df = pd.DataFrame(row_list, columns=column_names)
        print(df)
        
        for notice in con.notices:
            print(notice)

        cur.close()
        con.close()

    except (Exception, psycopg2.DatabaseError) as error:
            print(error)

    finally:
        if con is not None:
            con.close()

In [369]:
do_query("""
drop function if exists ai_med.ilosc_dni_lek_pacjent_(character, character, character);
""", 
commit=True)

function_def("""
create or replace function ai_med.ilosc_dni_lek_pacjent_(imieee char, nazwiskooo char, lek char)

returns table(
   id int,
   imie varchar,
   nazwisko varchar,
   data_rozpoczecie_brania_leku date,
   data_zakonczenia_brania_leku date,
   dni_brania_leku int
)

language plpgsql
as
$$

declare
   ilosc_pacjentow integer;

begin

   IF lek = 'ventolin'
      THEN
      return query
      SELECT 
      pacjent.id
      ,pacjent.imie
      ,pacjent.nazwisko
      ,"Aktualnie przyjmowane leki".data_rozpoczecia_ventolin__asthma
      ,"Aktualnie przyjmowane leki".data_zakonczenia_ventolin__asthma 
      ,coalesce("Aktualnie przyjmowane leki".data_zakonczenia_ventolin__asthma, cast(now() as date)) -
      "Aktualnie przyjmowane leki".data_rozpoczecia_ventolin__asthma as "dni_brania_ventolin"
      FROM ai_med.pacjent INNER JOIN ai_med."Aktualnie przyjmowane leki" 
      ON "Aktualnie przyjmowane leki".id = pacjent.id 
      WHERE pacjent.imie = imieee and pacjent.nazwisko = nazwiskooo;
   
   elsif lek = 'atorvastatin' 
      THEN
      return query
      SELECT 
      pacjent.id
      ,pacjent.imie
      ,pacjent.nazwisko
      ,"Aktualnie przyjmowane leki".data_rozpoczecia_atorvastatin_calcium__cholesterol
      ,"Aktualnie przyjmowane leki".data_zakonczenia_atorvastatin_calcium__cholesterol 
      ,coalesce("Aktualnie przyjmowane leki".data_zakonczenia_atorvastatin_calcium__cholesterol, cast(now() as date)) -
      "Aktualnie przyjmowane leki".data_rozpoczecia_atorvastatin_calcium__cholesterol as "dni_brania_ventolin"
      FROM ai_med.pacjent INNER JOIN ai_med."Aktualnie przyjmowane leki" 
      ON "Aktualnie przyjmowane leki".id = pacjent.id 
      WHERE pacjent.imie = imieee and pacjent.nazwisko = nazwiskooo;
   
   elsif lek = 'hydrochlorothiazide' 
      THEN
      return query
      SELECT
      pacjent.id
      ,pacjent.imie
      ,pacjent.nazwisko
      ,"Aktualnie przyjmowane leki".data_rozpoczecia_hydrochlorothiazide__wysokie_cisnienie_krwi
      ,"Aktualnie przyjmowane leki".data_zakonczenia_hydrochlorothiazide__wysokie_cisnienie_krwi 
      ,coalesce("Aktualnie przyjmowane leki".data_zakonczenia_hydrochlorothiazide__wysokie_cisnienie_krwi, cast(now() as date)) -
      "Aktualnie przyjmowane leki".data_rozpoczecia_hydrochlorothiazide__wysokie_cisnienie_krwi as "dni_brania_ventolin"
      FROM ai_med.pacjent INNER JOIN ai_med."Aktualnie przyjmowane leki" 
      ON "Aktualnie przyjmowane leki".id = pacjent.id 
      WHERE pacjent.imie = imieee and pacjent.nazwisko = nazwiskooo;
   
   ELSE 
      RAISE EXCEPTION 'Nie ma takiego leku';
   END IF;

end;
$$;
""",
commit = True)

In [370]:
function_tester("select * FROM ai_med.ilosc_dni_lek_pacjent_", "Rachelle", "Wheeler", "ventolin")

   id      imie nazwisko data_rozpoczecie_brania_leku  \
0   3  Rachelle  Wheeler                   2018-05-07   

  data_zakonczenia_brania_leku  dni_brania_leku  
0                   2020-08-04              820  


In [371]:
function_tester("select * FROM ai_med.ilosc_dni_lek_pacjent_", "Rachelle", "Wheeler", "atorvastatin")

   id      imie nazwisko data_rozpoczecie_brania_leku  \
0   3  Rachelle  Wheeler                   2018-02-16   

  data_zakonczenia_brania_leku  dni_brania_leku  
0                   2020-09-14              941  


In [372]:
function_tester("select * FROM ai_med.ilosc_dni_lek_pacjent_", "Rachelle", "Wheeler", "hydrochlorothiazide")

   id      imie nazwisko data_rozpoczecie_brania_leku  \
0   3  Rachelle  Wheeler                   2018-12-30   

  data_zakonczenia_brania_leku  dni_brania_leku  
0                   2021-02-27              790  


In [373]:
function_tester("select * FROM ai_med.ilosc_dni_lek_pacjent_", "Rachelle", "Wheler", "jakislek")

Nie ma takiego leku
CONTEXT:  PL/pgSQL function ai_med.ilosc_dni_lek_pacjent_(character,character,character) line 54 at RAISE



# TRIGGERS

In [220]:
def trigger_def(func, commit = False):
    try:
        con = psycopg2.connect(
                    host = "127.0.0.11",
                    database="ZSBD_p1",
                    user = "inzdanych123",
                    password = "inzdanych123",
                    port = "5432")

        cur = con.cursor()

        for notice in con.notices:
            print(notice)

        cur.execute(func)
        if commit:
            con.commit()

        cur.close()
        con.close()

    except (Exception, psycopg2.DatabaseError) as error:
            print(error)

    finally:
        if con is not None:
            con.close()

In [221]:
trigger_def("""
CREATE TRIGGER pacjent_insert_timestamp 
    AFTER
    INSERT 
    ON ai_med.pacjent
    FOR EACH ROW EXECUTE PROCEDURE ai_med.pacjent_insert_timestamp_func();
""",
commit=True)

trigger "pacjent_insert_timestamp" for relation "pacjent" already exists



In [222]:
function_def("""
CREATE OR REPLACE FUNCTION ai_med.pacjent_insert_timestamp_func() RETURNS TRIGGER AS $pacjent_table$
   BEGIN
      INSERT INTO ai_med.pacjent_insert_timestamp_table(id, ostatni_insert_danych) VALUES (NEW.id ,date_trunc('day', current_timestamp));
      RETURN NEW;
   END;
$pacjent_table$ LANGUAGE plpgsql;
""",
commit=True)

In [400]:
do_query("""
DROP TABLE IF EXISTS ai_med.pacjent_insert_timestamp_table
""", 
commit=True)
do_query("""
CREATE TABLE IF NOT EXISTS ai_med.pacjent_insert_timestamp_table
(
    id serial NOT NULL,
    ostatni_insert_danych date NOT NULL,
    ostatnie_powiadomienie_dla_MLE date
);
""", 
commit=True)

In [401]:
do_query("""
ALTER TABLE IF EXISTS ai_med.pacjent_insert_timestamp_table
    ADD FOREIGN KEY (id)
    REFERENCES ai_med.pacjent (id) MATCH SIMPLE
    ON UPDATE NO ACTION
    ON DELETE NO ACTION
    NOT VALID;
""", 
commit=True)

In [403]:
do_query("""
INSERT INTO ai_med.pacjent (imie, nazwisko, data_urodzenia) VALUES ('Dwight', 'Schrute', '1962-11-23');
""", 
commit=True,
print_out=True)
do_query("""
SELECT * FROM ai_med.pacjent where imie ='Dwight' and nazwisko = 'Schrute';
""", 
commit=True,
print_out=True)
do_query("""
SELECT * FROM ai_med.pacjent_insert_timestamp_table;
""", 
commit=True,
print_out=True)

'NoneType' object is not iterable
       id    imie nazwisko data_urodzenia                       created_at
0  110853  Dwight  Schrute     1962-11-23 2022-11-24 17:06:58.924089+01:00
1  110854  Dwight  Schrute     1962-11-23 2022-11-24 17:21:18.163307+01:00
2  110855  Dwight  Schrute     1962-11-23 2022-11-24 17:21:22.065947+01:00


       id ostatni_insert_danych ostatnie_powiadomienie_dla_mle
0  110854            2022-11-24                           None
1  110855            2022-11-24                           None




In [225]:
trigger_def("""
CREATE OR REPLACE TRIGGER message_to_ML_E
    BEFORE
    INSERT 
    ON ai_med.pacjent
    FOR EACH ROW EXECUTE PROCEDURE ai_med.message_to_ML_E_func();
""",
commit=True)

In [226]:
#TO W SUMIE POWINNO BYc NA KONIEC JAKIEJŚ TRANZAKCJI
#W SENSIE ŻEBY ZROBIĆ TRANZAKCJĘ, KTÓRA ZAJMIE SIĘ DAILY INSERTEM DO TABEL
function_def("""
CREATE OR REPLACE FUNCTION ai_med.message_to_ML_E_func() RETURNS TRIGGER AS $pacjent_table$
	declare
		liczba_wprowadzonych int;
	BEGIN
		SELECT count(*) into liczba_wprowadzonych from ai_med.pacjent_insert_timestamp_table 
		where ostatni_insert_danych = date_trunc('day', current_timestamp);
		
		IF liczba_wprowadzonych = 5
		THEN
		UPDATE ai_med.pacjent_insert_timestamp_table
		SET ostatnie_powiadomienie_dla_mle=date_trunc('minute', current_timestamp);
		raise notice 'Dzis zostalo wprowadzonych przynajmniej % nowych pacjentow, moze chcecie zrobic douczanie modelu?', 
		
		liczba_wprowadzonych + 1;
		
		END IF;
		RETURN NEW;
   	END;
$pacjent_table$ LANGUAGE plpgsql;
""",
commit=True)

#może jakoś że jak jest mniej insertów danego dnia niż x (5 powiedzmy) to rollback?

In [227]:
do_query("""
INSERT INTO ai_med.pacjent (imie, nazwisko, data_urodzenia) VALUES ('Paul', 'California', '2002-11-23');
""", 
commit=True,
print_out=True)

do_query("""
SELECT * FROM ai_med.pacjent_insert_timestamp_table;
""", 
commit=True,
print_out=True)

'NoneType' object is not iterable
       id ostatni_insert_danych ostatnie_powiadomienie_dla_mle
0  110830            2022-11-24                           None
1  110831            2022-11-24                           None
2  110832            2022-11-24                           None
3  110833            2022-11-24                           None




In [228]:
# do_query("""
# DROP TABLE IF EXISTS ai_med.pacjent_insert_timestamp_table
# """, 
# commit=True,
# print_out=True)

# do_query("""
# CREATE TABLE IF NOT EXISTS ai_med.pacjent_insert_timestamp_table
# (
#     id serial NOT NULL,
#     ostatni_insert_danych date NOT NULL,
#     ostatnie_powiadomienie_dla_MLE date
# );
# """, 
# commit=True,
# print_out=True)

# PROCEDURES

In [229]:
def procedure_def(func, commit = False):
    try:
        con = psycopg2.connect(
                    host = "127.0.0.11",
                    database="ZSBD_p1",
                    user = "inzdanych123",
                    password = "inzdanych123",
                    port = "5432")

        cur = con.cursor()

        for notice in con.notices:
            print(notice)

        cur.execute(func)
        if commit:
            con.commit()

        cur.close()
        con.close()

    except (Exception, psycopg2.DatabaseError) as error:
            print(error)

    finally:
        if con is not None:
            con.close()

# 1. Inserting data using a procedure GENEROWANIE danych za pomocą procedur?

# WYMAGANIA DO PROJEKTU NOTATKA TWOJA I TE PUNKTY CO SĄ POD SPODEM

In [454]:
procedure_def(
"""
create or replace procedure ai_med.select_all_pacjenty()
language plpgsql    
as $$
declare
    liczba_pacjentow numeric;

    liczba_danych_o_lekach numeric;
    liczba_danych_o_morfologii numeric;
    liczba_danych_o_objawach numeric;
    liczba_danych_o_obrazach numeric;
    liczba_danych_o_operacjach numeric;
    liczba_danych_o_notatkach numeric;
    liczba_danych_o_leczeniu numeric;
    --srednia_liczba_danych_per_pacjent numeric;
begin

    select ai_med.test1() into liczba_pacjentow;

    select count(*) into liczba_danych_o_lekach from ai_med."Aktualnie przyjmowane leki";
    select count(*) into liczba_danych_o_morfologii from ai_med.morfologia;
    select count(*) into liczba_danych_o_objawach from ai_med.objawy;
    select count(*) into liczba_danych_o_obrazach from ai_med.obrazy;
    select count(*) into liczba_danych_o_operacjach from ai_med.operacje;
    select count(*) into liczba_danych_o_notatkach from ai_med."Nazwa pliku z notatkami lekarskimi";
    select count(*) into liczba_danych_o_leczeniu from ai_med."leczenie_w_kierunku_kod_ICD_10";

    RAISE NOTICE 'W bazie jest juz: % pacjentow!! Kazdy pacjent ma o sobie % infomacji.', liczba_pacjentow , ((liczba_danych_o_lekach + liczba_danych_o_morfologii + liczba_danych_o_objawach + liczba_danych_o_obrazach + liczba_danych_o_operacjach + liczba_danych_o_notatkach + liczba_danych_o_leczeniu)/liczba_pacjentow)/7;

end;$$
""",
commit=True)

#MOŻE JAKIŚ RETURN TU (PAMIETAJ ŻE MA BYĆ 1 WIERSZ) I TO JAKO JAKIŚ WIDO DLA LEKARZA? CHODZI MI ŻEBY DOSTĘP DO TEGO DLA JAKIEGOŚ LEKARZA 

In [455]:
do_query("""
call ai_med.select_all_pacjenty();
""", 
commit=True)

NOTICE:  W bazie jest juz: 50115 pacjentow!! Kazdy pacjent ma o sobie 0.99770527786091988571 infomacji.





In [ ]:
#?JEŻELI PROCEDURA MOŻE PRZYJMOWAĆ JAKIEŚ ARGUMENTY? TO MOŻE PROCEDURA DO SPRAWDZANIA ZAKRESÓW NORM DLA JAKICHŚ PARAMETRÓW
#MOŻE TAKĄ FUNKCJE DODAĆ? 